In [25]:
# pip als Paketmanager
#! pip install -q pyscipopt
#! pip install pandas
#!pip install openpyxl

In [26]:
from pyscipopt import Model, quicksum

**Optimierungsmodell für den Kauf und Verkauf von Strom auf dem Strommarkt**

In [27]:
# Erstellen einer Modellinstanz
scip = Model()

**Indexmenge**

In [28]:
# Investition über Zeitraum j Tage
j = 3

H = [n for n in range(1, 24*j+1)]

**Batterie-Systemspezifikationen**

In [29]:
fixe_zykluskosten = True

wirkungsgrad_wechselrichter = 0.985
wirkungsgrad_laden = 0.975
round_trip_efficiency = 0.95
entlade_verlust = wirkungsgrad_laden - round_trip_efficiency

wirkungsgrad_systemeingang = wirkungsgrad_wechselrichter * wirkungsgrad_laden
wirkungsgrad_systemausgang = (1-(entlade_verlust / wirkungsgrad_laden)) * wirkungsgrad_wechselrichter


f_e = wirkungsgrad_systemeingang # Faktor Einkauf
f_v = wirkungsgrad_systemausgang # Faktor Verkauf


nennkapazität = 40 # MWh brutto
lademinimum = 0.2 # 20%
lademaximum = 1 # 100%
anfangsbestand = 0.5 # 50%

nettokapazität = zyklus = nennkapazität * (lademaximum - lademinimum) # MWh netto
zykluskosten = 1500 # € / zyklus
mwh_zykluskosten = zykluskosten / zyklus # € / MWh -> entweder geladen oder entladen

erlaubte_zyklen_pro_tag = 2

a = anfangsbestand * nennkapazität # MWh Anfangs- und Endbestand
u = lademinimum * nennkapazität # MWh Untergrenze Batteriekapazität
o = lademaximum * nennkapazität # MWh Obergrenze Batteriekapazität

c = 0.5 # nennkapazität / h


# Sicherstellen, dass unsere Faktoren für Systemeingang und -ausgang den multiplizierten Wirkungsgraden entspricht
print(wirkungsgrad_systemeingang*wirkungsgrad_systemausgang)
print(wirkungsgrad_wechselrichter*wirkungsgrad_wechselrichter*round_trip_efficiency)

0.92171375
0.92171375


**Vorhersagedaten**

In [30]:
import pandas as pd

prognose = pd.read_excel('Preisprognosen.xlsx')

p_h = {}

for h in range(1, 24+1):
    stundenprognose = prognose[prognose['Stunde'] == h]
    checksum = stundenprognose['Wahrscheinlichkeit'].sum()
    if checksum != 1:
        print('WARNING: Prognosedaten unvollständig')

    for i in range(0, j):
        p_h[h+i*24] = stundenprognose['Strompreis'].mean()

print (p_h)

{1: np.float64(81.52), 25: np.float64(81.52), 49: np.float64(81.52), 2: np.float64(72.24), 26: np.float64(72.24), 50: np.float64(72.24), 3: np.float64(68.16), 27: np.float64(68.16), 51: np.float64(68.16), 4: np.float64(66.64), 28: np.float64(66.64), 52: np.float64(66.64), 5: np.float64(66.8), 29: np.float64(66.8), 53: np.float64(66.8), 6: np.float64(71.04), 30: np.float64(71.04), 54: np.float64(71.04), 7: np.float64(88.56), 31: np.float64(88.56), 55: np.float64(88.56), 8: np.float64(92.68), 32: np.float64(92.68), 56: np.float64(92.68), 9: np.float64(80.92), 33: np.float64(80.92), 57: np.float64(80.92), 10: np.float64(61.08), 34: np.float64(61.08), 58: np.float64(61.08), 11: np.float64(43.08), 35: np.float64(43.08), 59: np.float64(43.08), 12: np.float64(30.88), 36: np.float64(30.88), 60: np.float64(30.88), 13: np.float64(22.44), 37: np.float64(22.44), 61: np.float64(22.44), 14: np.float64(15.96), 38: np.float64(15.96), 62: np.float64(15.96), 15: np.float64(12.52), 39: np.float64(12.52),

**Entscheidungsvariablen**

In [31]:
e_h={}
v_h={}

for h in H:
    e_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"e_{h}")
    v_h[h] = scip.addVar(vtype='C', lb=0, ub=None, name=f"v_{h}")

print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [e_1, v_1, e_2, v_2, e_3, v_3, e_4, v_4, e_5, v_5, e_6, v_6, e_7, v_7, e_8, v_8, e_9, v_9, e_10, v_10, e_11, v_11, e_12, v_12, e_13, v_13, e_14, v_14, e_15, v_15, e_16, v_16, e_17, v_17, e_18, v_18, e_19, v_19, e_20, v_20, e_21, v_21, e_22, v_22, e_23, v_23, e_24, v_24, e_25, v_25, e_26, v_26, e_27, v_27, e_28, v_28, e_29, v_29, e_30, v_30, e_31, v_31, e_32, v_32, e_33, v_33, e_34, v_34, e_35, v_35, e_36, v_36, e_37, v_37, e_38, v_38, e_39, v_39, e_40, v_40, e_41, v_41, e_42, v_42, e_43, v_43, e_44, v_44, e_45, v_45, e_46, v_46, e_47, v_47, e_48, v_48, e_49, v_49, e_50, v_50, e_51, v_51, e_52, v_52, e_53, v_53, e_54, v_54, e_55, v_55, e_56, v_56, e_57, v_57, e_58, v_58, e_59, v_59, e_60, v_60, e_61, v_61, e_62, v_62, e_63, v_63, e_64, v_64, e_65, v_65, e_66, v_66, e_67, v_67, e_68, v_68, e_69, v_69, e_70, v_70, e_71, v_71, e_72, v_72]


**Zielfunktion**

In [32]:
gewinn_kauf_verkauf = quicksum((p_h[h] * (v_h[h] - e_h[h])) for h in H)

if fixe_zykluskosten:
    zykluskosten = 3000 * j
else:
    zykluskosten = quicksum((e_h[h] * f_e * mwh_zykluskosten) for h in H)

scip.setObjective(gewinn_kauf_verkauf - zykluskosten, sense="maximize")
print(scip.getObjective())

Expr({Term(e_1): -81.52, Term(v_1): 81.52, Term(e_2): -72.24, Term(v_2): 72.24, Term(e_3): -68.16, Term(v_3): 68.16, Term(e_4): -66.64, Term(v_4): 66.64, Term(e_5): -66.8, Term(v_5): 66.8, Term(e_6): -71.04, Term(v_6): 71.04, Term(e_7): -88.56, Term(v_7): 88.56, Term(e_8): -92.68, Term(v_8): 92.68, Term(e_9): -80.92, Term(v_9): 80.92, Term(e_10): -61.08, Term(v_10): 61.08, Term(e_11): -43.08, Term(v_11): 43.08, Term(e_12): -30.88, Term(v_12): 30.88, Term(e_13): -22.44, Term(v_13): 22.44, Term(e_14): -15.96, Term(v_14): 15.96, Term(e_15): -12.52, Term(v_15): 12.52, Term(e_16): -18.44, Term(v_16): 18.44, Term(e_17): -34.88, Term(v_17): 34.88, Term(e_18): -56.56, Term(v_18): 56.56, Term(e_19): -78.44, Term(v_19): 78.44, Term(e_20): -105.6, Term(v_20): 105.6, Term(e_21): -144.8, Term(v_21): 144.8, Term(e_22): -125.12, Term(v_22): 125.12, Term(e_23): -101.36, Term(v_23): 101.36, Term(e_24): -85.28, Term(v_24): 85.28, Term(e_25): -81.52, Term(v_25): 81.52, Term(e_26): -72.24, Term(v_26): 72.

***Nebenbedingungen/ Restriktionen***

In [33]:
# Ladestand zur Stunde 0 = Ladestand zur Stunde 24, also Summe Lademenge und Entlademenge gleich
scip.addCons(quicksum(((e_h[h] * f_e) - (v_h[h] / f_v)) for h in H) == 0, name="Anfangs- und Endbestand gleich")

# Maximale Ladezyklen am pro Tag anhand der Einkaufsmenge (mit Faktor = Lademenge), alternativ anhand der Verkaufsmenge
scip.addCons(quicksum((e_h[h] * f_e) for h in H) <= (erlaubte_zyklen_pro_tag * nettokapazität * j), name="Maximale Ladezyklen pro Tag")

# Mindestladestand nicht unterschritten und Höchstladestand nicht überschritten
for h in H:
    H_t =  [n for n in range(1, h+1)]
    scip.addCons( (a + quicksum(((e_h[t] * f_e) - (v_h[t] / f_v)) for t in H_t)) >= u, name=f"Mindestladestand zum Zeitpunkt t={h}")
    scip.addCons( (a + quicksum(((e_h[t] * f_e) - (v_h[t] / f_v)) for t in H_t)) <= o, name=f"Maximalladestand zum Zeitpunkt t={h}")

# Lade- und Entladeleistung begrenzt (C-Rate)
for h in H:
    scip.addCons((e_h[h] * f_e) + (v_h[h] / f_v) <= c * nennkapazität, name=f"Lade-/Entladeleistung der Stunde h={h}")

print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Anfangs- und Endbestand gleich, Maximale Ladezyklen pro Tag, Mindestladestand zum Zeitpunkt t=1, Maximalladestand zum Zeitpunkt t=1, Mindestladestand zum Zeitpunkt t=2, Maximalladestand zum Zeitpunkt t=2, Mindestladestand zum Zeitpunkt t=3, Maximalladestand zum Zeitpunkt t=3, Mindestladestand zum Zeitpunkt t=4, Maximalladestand zum Zeitpunkt t=4, Mindestladestand zum Zeitpunkt t=5, Maximalladestand zum Zeitpunkt t=5, Mindestladestand zum Zeitpunkt t=6, Maximalladestand zum Zeitpunkt t=6, Mindestladestand zum Zeitpunkt t=7, Maximalladestand zum Zeitpunkt t=7, Mindestladestand zum Zeitpunkt t=8, Maximalladestand zum Zeitpunkt t=8, Mindestladestand zum Zeitpunkt t=9, Maximalladestand zum Zeitpunkt t=9, Mindestladestand zum Zeitpunkt t=10, Maximalladestand zum Zeitpunkt t=10, Mindestladestand zum Zeitpunkt t=11, Maximalladestand zum Zeitpunkt t=11, Mindestladestand zum Zeitpunkt t=12, Maximalladestand zum Zeitpunkt t=12, Mindestladestand zum Zeitpunkt t=13, Maximallades

**Berechnung der Lösung**

In [34]:
scip.setIntParam("display/verblevel", 5)  # Set verbosity level to 5


scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Gewinn) =', scip.getObjVal())
    for h in H:
        print("EINKAUF Stunde", h, " : " , scip.getVal(e_h[h]))
        print("Verkauf Stunde", h, " : " , scip.getVal(v_h[h]))

else:
    print('Problem hat keine Lösung')

LP Solver <Soplex 7.1.1>: barrier convergence tolerance cannot be set -- tolerance of SCIP and LP solver may differ
LP Solver <Soplex 7.1.1>: fastmip setting not available -- SCIP parameter has no effect
LP Solver <Soplex 7.1.1>: number of threads settings not available -- SCIP parameter has no effect
transformed problem has 144 variables (0 bin, 0 int, 0 impl, 144 cont) and 218 constraints
    218 constraints of type <linear>

original problem has 10872 active (34.633%) nonzeros and 10872 (34.633%) check nonzeros

feasible solution found by trivial heuristic after 0.0 seconds, objective value -9.000000e+03
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 288 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 1 del conss, 0 add conss, 288 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 0 del vars, 73 del conss, 0 add conss, 288 chg bounds, 0 chg sides, 0 chg coeffs, 0 u

In [35]:
einkauf_sum = sum(scip.getVal(e_h[h]) for h in H)
print(f'Es wurden {einkauf_sum} MWh eingekauft')
verkauf_sum = sum(scip.getVal(v_h[h]) for h in H)
print(f'Es wurden {verkauf_sum} MWh verkauft')

gesamtwirkungsgrad = (0.95*0.985*0.985)


if(gesamtwirkungsgrad != verkauf_sum / einkauf_sum):
    print(gesamtwirkungsgrad)
    print(verkauf_sum / einkauf_sum)
    print('CHECK: Wirkungsgrad stimmt nicht überein')
else:
    print('CHECK: Wirkungsgrad stimmt überein')

Es wurden 199.92190550566187 MWh eingekauft
Es wurden 184.27076923076925 MWh verkauft
CHECK: Wirkungsgrad stimmt überein


In [36]:
# Ergebnisse in Excel speichern

import pandas as pd
from openpyxl import load_workbook

# File and worksheet details
file_path = "Ergebnisse.xlsx"
if fixe_zykluskosten:
    sheet_name = "var1_fix"
else:
    sheet_name = "var1_var"

# Extract solution data from the solved SCIP model
solution_data = {
    "Hour": [h for h in H],
    "Einkauf": [round(scip.getVal(e_h[h]), 3) for h in H],
    "Verkauf": [round(scip.getVal(v_h[h]), 3) for h in H],
    "Gebotspreis": [
    "min" if round(scip.getVal(v_h[h]), 3) > 0 else "max" if round(scip.getVal(e_h[h]), 3) > 0 else "" 
    for h in H]
}

solution_df = pd.DataFrame(solution_data)

# Write the solution to the Excel file
try:
    # Load the workbook to check for existing worksheets
    workbook = load_workbook(file_path)
    if sheet_name not in workbook.sheetnames:
        # If the worksheet doesn't exist, create it
        print(f"Worksheet '{sheet_name}' not found. Creating it...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # If the worksheet exists, overwrite it
        print(f"Worksheet '{sheet_name}' found. Writing solution...")
        with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
            solution_df.to_excel(writer, sheet_name=sheet_name, index=False)
except FileNotFoundError:
    # If the file doesn't exist, create it and write the solution
    print(f"File '{file_path}' not found. Creating it and writing solution...")
    with pd.ExcelWriter(file_path, mode="w", engine="openpyxl") as writer:
        solution_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Solution written to '{file_path}' in the '{sheet_name}' worksheet.")


Worksheet 'var1_fix' found. Writing solution...
Solution written to 'Ergebnisse.xlsx' in the 'var1_fix' worksheet.
